# ABM

## Imports / Data Conversion

In [1]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import SimultaneousActivation, RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx
import panel as pn
import panel.widgets as pnw
import random
from tqdm import tqdm, trange
from time import sleep

data_path = '../' #set to wherever the data files are, will be used on every input

In [2]:
def kmtoNaut(km):
    return km / 1.852

In [48]:
ports = pd.read_csv((data_path +'ports.csv'))
data = pd.read_csv((data_path + 'distances.csv'))
origin = pd.read_csv((data_path + 'origin_ports.csv'))
route_blockages_horm = pd.read_csv((data_path + 'route_blockages_horm.csv'))

In [4]:
distances = data[["prev_port", "next_port", "distance"]]

In [5]:
distances.astype({'prev_port':'int64', 'next_port':'int64'}).dtypes

prev_port      int64
next_port      int64
distance     float64
dtype: object

In [6]:
N = data["next_port"].tolist()
N = list(set(N))
len(N)

2842

## Playground 

In [ ]:
# G = nx.from_pandas_edgelist(distances, "prev_port", "next_port",edge_attr= "distance",create_using=nx.MultiGraph())
# grid = NetworkGrid(G)

In [19]:
G = nx.MultiGraph()
G.add_nodes_from(N) #instatiate the ports as nodes of network
for i in range(len(distances)): #create bi-directional edges with an attribute for length 
       G.add_edge(distances.iloc[i][0], distances.iloc[i][1], length=distances.iloc[i][2])
grid = NetworkGrid(G) #Define Mesa Grid as the just created Network to allow for shipping only in routes

In [263]:
e = 0
for index, row in origin.iterrows():
    if pd.isna(ports.iloc[index]["INDEX_NO"]):
        e += 1

e

49

In [88]:
ports[ports["Unnamed: 0"]==286]
# ports.head()

,Unnamed: 0,PORT_NAME,INDEX_NO,coords
29,286,Hannan Port,NaN,"((135.31110022775667, 34.421924371272326),)"


In [ ]:
def next_dest(sample):
    try:

        overall_distance = list()
        itinerary = list()
        for i in range(len(sample)):
            distance = dict()
            for i in range(1,len(sample)):
                distance[sample[i]] = nx.dijkstra_path_length(G, sample[0] ,sample[i], weight='distance')
            next_stop = min(distance, key=distance.get)
            itinerary.append(nx.dijkstra_path(G, origin, next_stop, weight = 'distance'))
            overall_distance.append(next_stop)
            sample.pop(0)
            ind = sample.index(next_stop)
            sample.pop(ind)
            sample.insert(0, next_stop)
    except:
        pass
    flat_route = []
    for sublist in itinerary:
        for port in sublist:
            flat_route.append(port)
        
    return flat_route, sum(overall_distance)


## Model

In [52]:
class ShippingNetwork(Model):
    def __init__(self, N, distances, major_ports, S=2):
        N
        self.major_ports = major_ports
        self.num_ships = S
        self.distances = distances
        self.schedule = SimultaneousActivation(self)
        self.running = True

        self.G = nx.MultiGraph()
        self.G.add_nodes_from(N) #instatiate the ports as nodes of network
        
       
        for i in tqdm(range(len(distances)), desc="Building Network" ): #create bi-directional edges with an attribute for length 
            self.G.add_edge(distances.iloc[i][0], distances.iloc[i][1], length=distances.iloc[i][2])
            
        
        self.grid = NetworkGrid(self.G) #Define Mesa Grid as the just created Network to allow for shipping only in routes
        #create agents 
        Ships = []
       
        for i in tqdm(range(self.num_ships), desc="Placing Ships"):
            start_port = np.random.choice(self.major_ports["Ref"], size=None, replace=True,  p=self.major_ports["PROB"])
            destin_port = random.sample(self.G.nodes, k=1)[0]  # Sample the destination 
            #We sample the origin port from a list of the 50 biggest ports world, with the prob = TAU of the port / TAU of all origin ports for 2017
            a = Ship(i+1, self, self.G, start_port, destin_port, major_ports)
            self.schedule.add(a)
            #append to list of ships
            Ships.append(a)
        
            #place agent on origin node
            self.grid.place_agent(a, start_port)



        self.datacollector = DataCollector(
            model_reporters={"Graph":"G"},
            agent_reporters={"Position": "position", "Destination":"destination", "Itinerary":"itinerary", "Distance_Traveled":"distance_traveled", "Route":"current_route", "Route Changes":"route_chng" })
    
        #create ability to remove edges mid-model
    def network_change(self, change_type, route_chng):
        print(self.G.number_of_edges())
        if change_type == "add":
            self.G.add_edge(self.change_edge[0], self.change_edge[1], length=self.change_edge[2])
        if change_type == "remove":
            for index in range(len(route_chng)):
                for i in range(route_chng.iloc[index]['Num of edges']):
                    try:
                        self.G.remove_edge(route_chng.iloc[index]['prev_port'],route_chng.iloc[index]['next_port'])
                    except:
                        pass
            
        #update model with new grid
        self.grid = NetworkGrid(self.G)
        print(self.G.number_of_edges())

    def step(self, change_type='', change_edge=[]):
        #check network for changes
        if change_type != '' :
            self.grid, self.G = network_change(change_type, change_edge)

        self.schedule.step()     #Run each Agents
        self.datacollector.collect(self)

In [19]:
class Ship(Agent):
    def __init__(self, unique_id, model, G, start_port, destin_port, major_ports, s=13.0):
        super().__init__(unique_id, model)
        self.destination = destin_port
        self.state = 0 #0 for active, numbers > 0 for weeks that ships have to "wait" until arrival to port
        self.speed = s*24*1.852 #speed is given in knots, with 1 knot being 1 nautical mile per hour. Since the model works with distances in km, we convert here (1 nm = 1.852m)
        self.position = start_port
        self.distance_traveled = 0
        self.unique_id = unique_id
        self.G = G
        self.route_chng = 0
        self.routes = 0
        self.major_ports = major_ports

    
        #implement dijkstra to define shortest route
        try:
            self.init_route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            self.init_dist = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
        except:
            self.destination = random.sample(self.G.nodes, k=1)[0]
            self.init_route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            self.init_dist = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
            #include a way for capacity?
        

         #We keep a copy of the entire itinerary / distance traveled
        self.current_route, self.current_dist = self.init_route, self.init_dist  #For comparison & navigational purposes, we use current route & distance
        self.next_position = self.current_route[1]
        self.itinerary = [self.position]
        self.step_size = self.ident_distance()


    def routing(self):
        #implement dijkstra to define shortest route
        try:
            route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            travel_distance = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
        except:
            self.destination = random.sample(self.G.nodes, k=1)[0]
            route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            travel_distance = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
        #include a way for capacity?
        return route, travel_distance


    def move(self):
        self.next_position = self.current_route[1]
        self.step_size = self.ident_distance() #look up the distance between two cities 
        self.state = self.step_size / self.speed #change state to step amount
        self.current_dist = self.current_dist - self.step_size #adjust current distance minus the distance traveled in the next step
        self.model.grid.move_agent(self, self.next_position) #move the agent
        self.current_route.remove(self.current_route[0]) #remove the next step from the itinerary
        self.position = self.next_position
        if len(self.current_route) == 1:
            self.next_position = self.current_route[0] 
        else:
            self.next_position = self.current_route[1] #update current route

    def ident_distance(self): #look up the distance of the current step
        return self.G.get_edge_data(self.position, self.next_position)[0]['length']
    
    def reroute(self):
        self.routes += 1
        if self.position in self.major_ports["Ref"].values: # We re-route in such a way that ships that have either an origin port or a destination port that is a major port
            self.destination = random.sample(self.G.nodes, k=1)[0]
        else:
            self.destination = np.random.choice(self.major_ports["Ref"], size=None, replace=True,  p=self.major_ports["PROB"])
        
        self.current_route, self.current_dist = self.routing()
        self.state = 3

    

    def step(self):
        self.state = self.state - 1 #'move' ships by one weeks progress
        if self.state <= 0.000: #ships that are en-route to the node they are going to next do not move / perform other activities
            self.distance_traveled += self.step_size #ship has arrived at port, let's add the distance traveled to their 
           
            #add the current position to itinerary
            if self.position != self.destination: #if current stop is not the final stop
                new_route, new_distance = self.routing() #perform a new routing to compare against current routing
                
                if new_route == self.current_route: #if current routing is the same as new, just move (default case)
                    # print("default case")
                    self.move()
                    self.itinerary.append(self.position)
        
            # THIS CURRENTLY ONLY CHANGES THE ROUTE IF THE NEXT STEP IS BLOCKED
                elif new_distance > self.current_dist: #if current route is shorter than newly calculated route, check for obstructions
                    # print('reroute: current route shorter than new route')
                    if not has_path(self.G, self.position, self.next_position): 
                        self.current_route = new_route
                        self.current_dist = new_distance
                        self.route_chng += 1
                    self.move()
                    self.itinerary.append(self.position)
                
                
                else: # final option is that current route is longer than new route (think Suez reopening after a while), here, we just take the new option
                    # print('reroute: current route longer than new route')
                    self.current_route = new_route
                    self.current_dist = new_distance
                    self.route_chng += 1
                    self.move()
                    self.itinerary.append(self.position)
            
            else: #if ship is arrived at final position, get a new route, and start back
                # print('arrival')
                self.reroute()
                self.itinerary.append(self.position)
        # print("Ship: {}, Source: {}, Destination: {}, Position: {},  Next Stop {}, Time until next Stop {}".format(self.unique_id, self.itinerary[0], self.destination, self.position, self.next_position, self.state ))

    
    # def collect_time

    # def collect_costs

## Model Instances

In [73]:
test = route_blockages_horm.iloc[0:20]
model = ShippingNetwork(N, distances, origin, 1)

steps = 6
for i in trange(steps, desc="Stepping Model"):
    model.step()
model.network_change("remove", route_blockages_horm)

for i in trange(6, desc="Stepping after change", delay=0.5):
    model.step()

# route_blockages_horm

agent_state = model.datacollector.get_agent_vars_dataframe()


Stepping Model: 100%|██████████| 6/6 [00:01<00:00,  5.32it/s]
56154
54481


ValueError: list.remove(x): x not in list

In [70]:
agent_state = model.datacollector.get_agent_vars_dataframe()
a

,,Position,Destination,Itinerary,Distance_Traveled,Route,Route Changes
Step,AgentID,,,,,,
1,1,22735.0,3812,"[3728.0, 22735.0, 2667.0, 6869.0]",9.722991,"[6869.0, 2052.0, 2111.0, 1592.0, 1761.0, 3026....",0
2,1,2667.0,3812,"[3728.0, 22735.0, 2667.0, 6869.0]",19.445982,"[6869.0, 2052.0, 2111.0, 1592.0, 1761.0, 3026....",0
3,1,2667.0,3812,"[3728.0, 22735.0, 2667.0, 6869.0]",19.445982,"[6869.0, 2052.0, 2111.0, 1592.0, 1761.0, 3026....",0
4,1,6869.0,3812,"[3728.0, 22735.0, 2667.0, 6869.0]",796.007266,"[6869.0, 2052.0, 2111.0, 1592.0, 1761.0, 3026....",0
5,1,6869.0,3812,"[3728.0, 22735.0, 2667.0, 6869.0]",796.007266,"[6869.0, 2052.0, 2111.0, 1592.0, 1761.0, 3026....",0
6,1,6869.0,3812,"[3728.0, 22735.0, 2667.0, 6869.0]",796.007266,"[6869.0, 2052.0, 2111.0, 1592.0, 1761.0, 3026....",0
7,1,6869.0,3812,"[3728.0, 22735.0, 2667.0, 6869.0]",796.007266,"[6869.0, 2052.0, 2111.0, 1592.0, 1761.0, 3026....",0
8,1,6869.0,3812,"[3728.0, 22735.0, 2667.0, 6869.0]",796.007266,"[6869.0, 2052.0, 2111.0, 1592.0, 1761.0, 3026....",0
9,1,6869.0,3812,"[3728.0, 22735.0, 2667.0, 6869.0]",796.007266,"[6869.0, 2052.0, 2111.0, 1592.0, 1761.0, 3026....",0


In [68]:
agent_state.to_csv('../test_out.csv')

138038.67166337415